# Automatic Jupyter Notebook : Scenario 2
This is an auto-generated notebook generated using the classifAI DSL on : 22/01/2023 13:29:44

## Selection of data

In [ ]:
import os
import numpy as np
from PIL import Image

In [ ]:
X = []
Y = []
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
classes_count = {'0': 5923, '1': 6742, '2': 5958, '3': 6131, '4': 5842, '5': 5421, '6': 5918, '7': 6265, '8': 5851, '9': 5949}
for class_ in classes:
	count = 0
	for file in os.listdir(f'input_data/' + class_):
		if count == classes_count[class_]:
			break
		X.append(np.array(Image.open(f'input_data/' + class_ + '/' + file)))
		Y.append(class_)
		count += 1
X=np.array(X)
Y=np.array(Y)
print("X shape :",X.shape)
print("Y shape :",Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Preprocessing of data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.fit_transform(Y_test)

## Transformation of data

### Normalization of data

In [ ]:
# Making sure that the values are float so that we can get decimal points after division
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
X_train = X_train / 255
X_test = X_test / 255
print("X train shape :",X_train.shape)
print("X test shape :",X_test.shape)

### Reshaping of data

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
print("X train shape :",X_train.shape)
print("X train shape :",X_test.shape)

## Data Mining

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

### Keeping track of the models and the metrics

In [ ]:
models_metrics = dict()

### Classifier : rank 1 - number 1

Reshaping the data of this classifier

In [ ]:
X_train_save = X_train
X_test_save = X_test
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
print("X train shape :",X_train.shape)
print("X test shape :",X_test.shape)

Creating the model

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(48, activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(5, activation="relu"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Training and evaluating the model

In [ ]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=32)
score = model.evaluate(X_test, Y_test)
print("Loss : ", score[0])
print("Accuracy : ", score[1])
models_metrics["1-1"] = [history, score, model]
X_train = X_train_save
X_test = X_test_save

### Classifier : rank 1 - number 2

Reshaping the data of this classifier

In [ ]:
X_train_save = X_train
X_test_save = X_test
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
print("X train shape :",X_train.shape)
print("X test shape :",X_test.shape)

Creating the model

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(48, activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(5, activation="relu"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Training and evaluating the model

In [ ]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=32)
score = model.evaluate(X_test, Y_test)
print("Loss : ", score[0])
print("Accuracy : ", score[1])
models_metrics["1-2"] = [history, score, model]
X_train = X_train_save
X_test = X_test_save

## Comparison of classifiers

### Importing libraries

In [ ]:
import matplotlib.pyplot as plt

### Plotting the metrics

#### Plot of the accuracy

In [ ]:
# Plot accuracy of the classifiers
for key, value in models_metrics.items():
	if value[1][1] >= 0.5 and value[1][0] <= 0.5:
		history = value[0]
		plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(models_metrics.keys(), loc='upper left')
plt.show()

#### Plot of the loss

In [ ]:
# Plot loss of the classifiers
for key, value in models_metrics.items():
	if value[1][1] >= 0.5 and value[1][0] <= 0.5:
		history = value[0]
		plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(models_metrics.keys(), loc='upper left')
plt.show()

## Deploying the best model

### Finding the best model according to the accuracy

In [ ]:
best_model = None        
best_accuracy = 0        
for key, value in models_metrics.items():            
	accuracy = value[1][1]            
	if accuracy > best_accuracy:                
		best_accuracy = accuracy                
		best_model = value[2]        
print(f"Best model accuracy: {best_accuracy}")

### Saving the best model

In [ ]:
best_model.save("../prediction-app/model/model.h5")        
print("The best model has been saved in the directory ../prediction-app/model/model.h5")

### Setting the name of the application

In [ ]:
with open("../prediction-app/.env", "w") as file:        
	file.write("APP_NAME=\"My Prediction App Scenario2\"")

### Instructions for running the application

1. Open the terminal and go to the directory ../prediction-app

2. Run the command: steamlit run app.py

3. Open the browser and go to the address: http://localhost:8501